# 构建一个简单的Agent

zero-shot-react-description

In [17]:
import os
from langchain_community.llms.tongyi import Tongyi
from langchain.agents import load_tools, initialize_agent
from langchain.agents.agent_types import AgentType

# 加载llm
llm = Tongyi(model="qwen-plus", dashscope_api_key=os.getenv("DASHSCOPE_API_KEY")) # type: ignore

# 加载工具
# serpapi是google开发的一个基于google search的api
tools = load_tools(["serpapi"])

# 初始化agent
agent = initialize_agent(tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# 调用agent
# verbose参数的主要作用是: 是否打开详细的日志输出
agent("What's the date today? What great events have taken place today in history?")

/tmp/ipykernel_2613/1792568364.py:14: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)




> Entering new AgentExecutor chain...
I need to determine today's date first and then search for significant historical events that occurred on this date.
Action: Search
Action Input: What is the date today?
Observation: Tuesday, June 17, 2025
Thought:Now that I know the date, I should search for significant historical events that took place on June 17.
Action: Search
Action Input: Significant historical events on June 17
Observation: ["Historical Events on June 17 · Vlad's Assassination Attempt · Battle of Deptford Bridge · Cardinal Fischer Trialed for Treason · Francis Drake Lands in California.", '1972 The arrest of five White House operatives sets off the Watergate scandal · 1944 Iceland becomes a republic · 1940 The three Baltic states fall under Soviet ...', 'Battle of Bunker Hill begins · Native Americans score victory at the Battle of the Rosebud · Future President Hoover caught in Boxer Rebellion · France signals ...', 'On this day in 1994, American gridiron football hero OJ

{'input': "What's the date today? What great events have taken place today in history?",
 'output': "Today's date is June 17, 2025. Some great events that have taken place on June 17 in history include:\n\n- 1631: Mumtaz Mahal dies during childbirth.\n- 1665: Battle of Montes Claros, where Portugal secured independence from Spain.\n- 1775: Battle of Bunker Hill during the American Revolutionary War.\n- 1885: The Statue of Liberty arrives in New York City as a gift from France.\n- 1928: Amelia Earhart becomes the first woman to fly across the Atlantic Ocean.\n- 1940: The three Baltic states fall under Soviet control.\n- 1944: Iceland becomes a republic.\n- 1972: The Watergate scandal begins with the arrest of five White House operatives."}

# 文本总结

In [8]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.llms.tongyi import Tongyi


In [26]:
# 加载非结构化文本加载器
loader = UnstructuredFileLoader("./timemachine.txt")

# 将文本加载入内存
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 0
)

# 文本分割chunks
split_documents = text_splitter.split_documents(documents)

# 调用语言模型
llm = Tongyi(model="qwen-plus", dashscope_api_key=os.getenv("DASHSCOPE_API_KEY")) # type: ignore

chain = load_summarize_chain(llm=llm, chain_type="refine", verbose=True)
chain.run(split_documents[:10])



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"Project Gutenberg's The Time Machine, by H. G. (Herbert George) Wells

This eBook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever.  You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.net

Title: The Time Machine

Author: H. G. (Herbert George) Wells

Release Date: October 2, 2004 [EBook #35] [Last updated: October 3, 2014]

Language: English"


CONCISE SUMMARY:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Your job is to produce a final summary.
We have provided an existing summary up to a certain point: This is a free eBook of H. G. Wells' "The Time Machine," available through Project Gutenberg. It can be copied, shared, or reused under the terms of the Project Gutenberg L

'This is a free eBook of H. G. Wells\' "The Time Machine," available through Project Gutenberg. It can be copied, shared, or reused under the terms of the Project Gutenberg License, with no cost and minimal restrictions. Originally published in 1898, the eBook version was released in English on October 2, 2004 (EBook #35), and last updated on October 3, 2014. The story begins with the Time Traveller, a passionate and animated individual, introducing the concept of time travel to a group of friends during an intimate gathering. Set against the backdrop of a warm, luxurious after-dinner atmosphere, the scene establishes the foundation for the Time Traveller\'s extraordinary journey through time. During this discussion, the Time Traveller challenges universally accepted ideas, such as traditional geometry, by explaining that dimensions like length, breadth, and thickness are insufficient to fully describe reality without considering duration. He emphasizes that any real body must have ext

# 使用langchain构建一个初级的RAG系统

In [ ]:
from langchain.vectorstores.chroma import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.llms.tongyi import Tongyi
from langchain_community.embeddings.dashscope import DashScopeEmbeddings
from langchain.document_loaders import UnstructuredFileLoader
from langchain.chains.retrieval_qa.base import RetrievalQA

loader = UnstructuredFileLoader(file_path="./斗破苍穹.txt")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_docs = text_splitter.split_documents(documents=documents)

llm = Tongyi(model="qwen-plus")
embedding_func = DashScopeEmbeddings(model="text-embedding-v1")
# from_documents函数会调用embedding_fuc的 `embed_documents` 函数, 返回一个Chroma对象 `doc_search`
# doc_search可以使用一个方法 `as_retriever` 来返回一个 `VectorStoreRetriever` 类型
doc_search = Chroma.from_documents(split_docs, embedding=embedding_func)

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=doc_search.as_retriever(), return_source_documents=True)
result = qa({"query": "这篇文章主要说的什么内容"} )

In [ ]:
print(result['result'])
for i in result['source_documents']:
    print(i)

这篇文章主要描述了少年萧炎在家族中的测验中只展现出“斗之力三段”的低级水平，与他昔日天才的名声形成巨大反差，遭到了周围人的嘲笑和轻视。与此同时，另一位家族成员萧媚则表现出色，达到了“斗之气七段”的高级水平，受到众人称赞。文章通过对比展现了萧炎目前的困境以及他在斗气大陆上面临的挑战，包括对力量境界的追求和材料收集的困难，体现了斗气大陆中实力至上的残酷现实。
page_content='第一章 陨落的天才

“斗之力，三段”

望着测验魔石碑上面闪亮得甚至有些刺眼的五个大字，少年面无表情，唇角有着一抹自嘲，紧握的手掌，因为大力，而导致略微尖锐的指甲深深的刺进了掌心之中，带来一阵阵钻心的疼痛

“萧炎，斗之力，三段级别：低级”测验魔石碑之旁，一位中年男子，看了一眼碑上所显示出来的信息，语气漠然的将之公布了出来

中年男子话刚刚脱口，便是不出意外的在人头汹涌的广场上带起了一阵嘲讽的骚动武动乾坤。

“三段嘿嘿，果然不出我所料，这个“天才”这一年又是在原地踏步”

“哎，这废物真是把家族的脸都给丢光了。”

“要不是族长是他的父亲，这种废物，早就被驱赶出家族，任其自生自灭了，哪还有机会待在家族中白吃白喝。”

“唉，昔年那名闻乌坦城的天才少年，如今怎么落魄成这般模样了啊”

“谁知道呢，或许做了什么亏心事，惹得神灵降怒了吧”

周围传来的不屑嘲笑以及惋惜轻叹，落在那如木桩待在原地的少年耳中，恍如一根根利刺狠狠的扎在心脏一般，让得少年呼吸微微急促武动乾坤。

少年缓缓抬起头来，露出一张有些清秀的稚嫩脸庞，漆黑的眸子木然的在周围那些嘲讽的同龄人身上扫过，少年嘴角的自嘲，似乎变得更加苦涩了武动乾坤。

“这些人，都如此刻薄势力吗或许是因为三年前他们曾经在自己面前露出过最谦卑的笑容，所以，如今想要讨还回去吧”苦涩的一笑，萧炎落寞的转身，安静的回到了队伍的最后一排，孤单的身影，与周围的世界，有些格格不入武动乾坤。

“下一个，萧媚”

听着测验人的喊声，一名少女快速的人群中跑出，少女刚刚出场，附近的议论声便是小了许多，一双双略微火热的目光，牢牢的锁定着少女的脸颊

少女年龄不过十四左右，虽然并算不上绝色，不过那张稚气未脱的小脸，却是蕴含着淡淡的妩媚，清纯与妩媚，矛盾的集合，让得她成功的成为了全场瞩目的焦点

少女快步上前，小手轻车熟路的触摸着漆黑的魔石碑，然后缓缓闭上眼睛

在少女闭眼